---
title: "Assignment 3"
subtitle: "Big Data Visualization on Scale"
author:
  - name: "Othmane Elouardi"
    affiliation: "Boston University"
date: today
number-sections: true
format:
  html:
    theme: [lux, styles.scss]
    toc: true
    toc-depth: 3
    code-fold: false       # hide fold UI
    code-tools: false      # hide copy/run UI
    code-copy: false
    highlight-style: github
    title-block-style: default
    title-block-banner: true
    df-print: paged
execute:
  enabled: true            # <-- must be in the header
  echo: false
  warning: false
  message: false
  freeze: auto
---

---


# Load the Dataset

&emsp;&emsp;I used **PySpark** to bring the `lightcast_data.csv` file into a Spark DataFrame. After loading the data, I checked the schema to confirm that the column names were correct and could be used reliably in the later analysis. For each visualization, I customized the color schemes, fonts, and styles rather than relying on defaults, and I added a short explanation under each figure to highlight the main insights.  

---

## Loading Dataset With PySpark

&emsp;• Initialized a Spark session  
&emsp;• Loaded the `lightcast_data.csv` file into a DataFrame  
&emsp;• Displayed the schema and a sample of the dataset as a quick check  



```python
# Imports
import pandas as pd
import plotly.express as px
import plotly.io as pio
from pyspark.sql import SparkSession
import re
import numpy as np
import plotly.graph_objects as go
from pyspark.sql.functions import col, split, explode, regexp_replace, transform, when
from pyspark.sql import functions as F
from pyspark.sql.functions import col, monotonically_increasing_id
```

```python
# Spark session
spark = (
    SparkSession.builder
    .appName("LightcastData")
    .getOrCreate()
)

# Load the CSV
csv_path = "data/lightcast_job_postings.csv"

df = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("multiLine", "true")
        .option("escape", "\"")
        .csv(csv_path)
)

# Temp view for SQL
df.createOrReplaceTempView("job_postings")

# Quick diagnostics 
#print("\n--- Spark & Data Quick Check ---")
#print("Spark version:", spark.version)
#print("Row count (sampled below):")
#df.show(5, truncate=False)    
# df.printSchema()            
```



## DATA CLEANING

In this section, we performed several cleaning steps to ensure the dataset is consistent and ready for analysis:

- Cast salary/experience columns to numeric  
- Compute medians (via `approxQuantile`) for imputation  
- Create `Average_Salary` with sensible fallbacks  
- Clean `EDUCATION_LEVELS_NAME` (remove newlines)  
- Derive a simplified `REMOTE_GROUP` (`Remote`, `Hybrid`, `Onsite`)  

---

### Casting

```python
# Cast target columns to numeric (DoubleType)
from pyspark.sql.types import DoubleType

numeric_casts = {
    "SALARY_FROM": DoubleType(),
    "SALARY_TO":   DoubleType(),
    "SALARY":      DoubleType(),
    "MIN_YEARS_EXPERIENCE": DoubleType(),
    "MAX_YEARS_EXPERIENCE": DoubleType()
}

df = df.select(*[
    F.col(c).cast(numeric_casts[c]).alias(c) if c in numeric_casts else F.col(c)
    for c in df.columns
])
```

### Medians

```python
# Helper to compute median with approxQuantile
def median_of(colname):
    vals = df.filter(F.col(colname).isNotNull()) \
             .approxQuantile(colname, [0.5], 0.001)
    return float(vals[0]) if vals else None

median_from   = median_of("SALARY_FROM")
median_to     = median_of("SALARY_TO")
median_salary = median_of("SALARY")

print("Medians:", median_from, median_to, median_salary)
```




#### Medians (output)

In [ ]:
#| echo: false
#| warning: false

from pyspark.sql import SparkSession, functions as F

# Ensure Spark & a DataFrame named `df` are available
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("LightcastData").getOrCreate()

try:
    df  # reuse if already defined earlier
except NameError:
    if any(t.name == "job_postings_clean" for t in spark.catalog.listTables()):
        df = spark.table("job_postings_clean")
    else:
        df = (
            spark.read
            .option("header", "true").option("inferSchema", "true")
            .option("multiLine", "true").option("escape", "\"")
            .csv("data/lightcast_job_postings.csv")
        )

# Helper to compute median with approxQuantile (returns None if column missing)
def median_of(colname):
    if colname not in df.columns:
        return None
    vals = df.filter(F.col(colname).isNotNull()).approxQuantile(colname, [0.5], 0.001)
    return float(vals[0]) if vals else None

# Compute medians
median_from   = median_of("SALARY_FROM")
median_to     = median_of("SALARY_TO")
median_salary = median_of("SALARY")

# Nicely formatted output (only text appears on page)
from IPython.display import display, HTML
html = f"""
<div style="font-size:1rem; line-height:1.6;">
  <strong>Medians</strong>
  <ul style="margin:0.25rem 0 0 1.25rem;">
    <li>SALARY_FROM: {median_from:,.0f}</li>
    <li>SALARY_TO: {median_to:,.0f}</li>
    <li>SALARY: {median_salary:,.0f}</li>
  </ul>
</div>
"""
display(HTML(html))

### Average Salary

```python
# Choose a global fallback for missing salary values
fallback = next(x for x in [median_salary, median_from, median_to] if x is not None)

df = df.withColumn(
    "Average_Salary",
    F.when(F.col("SALARY_FROM").isNotNull() & F.col("SALARY_TO").isNotNull(),
           (F.col("SALARY_FROM") + F.col("SALARY_TO")) / 2.0
    ).when(F.col("SALARY").isNotNull(), F.col("SALARY")
    ).when(F.col("SALARY_FROM").isNotNull(), F.col("SALARY_FROM")
    ).when(F.col("SALARY_TO").isNotNull(), F.col("SALARY_TO")
    ).otherwise(F.lit(fallback))
)

# fill missing bounds for completeness
df = df.fillna({"SALARY_FROM": median_from, "SALARY_TO": median_to})
```


#### Average Salary (output)


In [ ]:
#| echo: false
#| warning: false

from pyspark.sql import SparkSession, functions as F
from IPython.display import display, HTML

# Ensure Spark & df exist
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("LightcastData").getOrCreate()

try:
    df  # reuse if defined earlier
except NameError:
    if any(t.name == "job_postings_clean" for t in spark.catalog.listTables()):
        df = spark.table("job_postings_clean")
    else:
        df = (
            spark.read
            .option("header", "true").option("inferSchema", "true")
            .option("multiLine", "true").option("escape", "\"")
            .csv("data/lightcast_job_postings.csv")
        )

# Helper to compute median if we need to rebuild fallback
def _median_of(colname):
    if colname not in df.columns:
        return None
    vals = df.filter(F.col(colname).isNotNull()) \
             .approxQuantile(colname, [0.5], 0.001)
    return float(vals[0]) if vals else None

# Use existing `fallback` if present; otherwise rebuild it exactly like before
try:
    avg_value = float(fallback)
except NameError:
    median_from   = _median_of("SALARY_FROM")
    median_to     = _median_of("SALARY_TO")
    median_salary = _median_of("SALARY")
    # same selection rule as your pipeline
    for x in (median_salary, median_from, median_to):
        if x is not None:
            avg_value = x
            break
    else:
        avg_value = None  # if everything missing

# Render result (text only)
if avg_value is None:
    display(HTML("<em>Average Salary could not be determined from the available fields.</em>"))
else:
    display(HTML(f"""
    <div style="font-size:1rem; line-height:1.6;">
      <strong>Average Salary</strong>: {avg_value:,.0f} USD
    </div>
    """))

### Clean Education Field

```python
# Remove \r and \n, then trim
df = df.withColumn(
    "EDUCATION_LEVELS_NAME",
    F.trim(
        F.regexp_replace(
            F.regexp_replace(F.col("EDUCATION_LEVELS_NAME"), r"\r", " "),
            r"\n", " "
        )
    )
)
```


#### Clean Education Field (output)

In [ ]:
#| echo: false
#| warning: false

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import col
from IPython.display import display, HTML

# Ensure Spark & df exist
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("LightcastData").getOrCreate()

try:
    df
except NameError:
    if any(t.name == "job_postings_clean" for t in spark.catalog.listTables()):
        df = spark.table("job_postings_clean")
    else:
        df = (
            spark.read
            .option("header", "true").option("inferSchema", "true")
            .option("multiLine", "true").option("escape", "\"")
            .csv("data/lightcast_job_postings.csv")
        )

colname = "EDUCATION_LEVELS_NAME"

if colname not in df.columns:
    display(HTML("<em>EDUCATION_LEVELS_NAME column not found — nothing to summarize.</em>"))
else:
    total_rows = df.count()
    # blanks or nulls *after* cleaning step
    blanks = (
        df.filter( (col(colname).isNull()) | (F.trim(col(colname)) == "") )
          .count()
    )

    # Top 10 education labels
    top_pdf = (
        df.groupBy(colname).count()
          .orderBy(F.desc("count"))
          .limit(10)
          .toPandas()
    )

    # Build HTML
    header = f"""
    <div style="font-size:1rem; line-height:1.6; margin-bottom:.5rem;">
      <strong>Education Field Summary (post-clean)</strong><br/>
      Total rows: {total_rows:,} • Blank/Null: {blanks:,}
    </div>
    """

    # Simple table
    if len(top_pdf) > 0:
        # pretty format
        top_pdf.columns = ["Education Level", "Count"]
        top_pdf["Count"] = top_pdf["Count"].map(lambda x: f"{x:,}")
        table_html = top_pdf.to_html(index=False, border=0)
        display(HTML(header + table_html))
    else:
        display(HTML(header + "<em>No non-empty education values found.</em>"))

### Remote Group

```python
# Normalize to Remote / Hybrid / Onsite
df = df.withColumn(
    "REMOTE_GROUP",
    F.when(F.col("REMOTE_TYPE_NAME") == "Remote", "Remote")
     .when(F.col("REMOTE_TYPE_NAME") == "Hybrid", "Hybrid")
     .otherwise("Onsite")
)
```




### Final Check

```python
df.createOrReplaceTempView("job_postings_clean")

print("Rows retained:", df.count())

df.select("Average_Salary","SALARY","EDUCATION_LEVELS_NAME",
          "REMOTE_TYPE_NAME","REMOTE_GROUP",
          "MIN_YEARS_EXPERIENCE","MAX_YEARS_EXPERIENCE") \
  .show(5, truncate=False)
```











# Salary Distribution by Industry and Employment Type

In this section, we explore salary variation across industries and employment types:

- **Filter the dataset**
  - Remove records where **salary is missing or zero**.

- **Aggregate / organize**
  - Group by **NAICS industry** using `NAICS2_NAME`.
  - Keep salary values from `SALARY_FROM` (or `Average_Salary` if preferred).
  - Separate distributions by **employment type** using `EMPLOYMENT_TYPE_NAME`.

- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
    - **Color / group** = `EMPLOYMENT_TYPE_NAME`
  - Customize axis labels, tick angle, margins, and figure size for readability.


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: Salary Distribution by Industry and Employment Type

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
import plotly.express as px
from IPython.display import HTML, display

# Reuse Spark session if already started, otherwise create new
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Reuse cleaned table if exists, else reload CSV
if any(t.name == "job_postings_clean" for t in spark.catalog.listTables()):
    df_use = spark.table("job_postings_clean")
else:
    df_use = (
        spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("multiLine", "true")
        .option("escape", "\"")
        .csv("data/lightcast_job_postings.csv")
        .withColumn("SALARY_FROM", F.col("SALARY_FROM").cast(DoubleType()))
    )

# Filter and select relevant columns
plot_sdf = (
    df_use.filter((col("SALARY_FROM").isNotNull()) & (col("SALARY_FROM") > 0))
          .select("NAICS2_NAME", "SALARY_FROM", "EMPLOYMENT_TYPE_NAME")
)

# Convert to pandas for Plotly
plot_df = plot_sdf.toPandas()
plot_df["NAICS2_NAME"] = plot_df["NAICS2_NAME"].astype(str).str.replace(r"\s+", " ", regex=True)

# Build Plotly box plot
fig = px.box(
    plot_df,
    x="NAICS2_NAME",
    y="SALARY_FROM",
    color="EMPLOYMENT_TYPE_NAME",
    points="outliers",
    labels={
        "NAICS2_NAME": "Industry (NAICS 2-Digit)",
        "SALARY_FROM": "Salary",
        "EMPLOYMENT_TYPE_NAME": "Employment Type",
    },
    title="Salary Distribution by Industry and Employment Type",
)

# Make figure larger and more readable
fig.update_layout(
    xaxis_title=None,
    yaxis_title="Salary",
    margin=dict(l=40, r=40, t=80, b=350),  # more bottom margin
    boxmode="group",
    width=2000,   # much wider
    height=800    # taller
)
fig.update_xaxes(tickangle=45)


# Render as HTML so it shows on Quarto site
display(HTML(fig.to_html(include_plotlyjs="cdn", full_html=False)))

- **Explanation**:
  “ This box plot shows salary variations by industry and employment type. Full-time positions tend to have higher median salaries and greater variability compared to part-time roles. Outliers, particularly in Construction, Health Care, and Finance, highlight some extremely high or low salaries, providing a clear view of the salary landscape across sectors.”








# Salary Analysis by ONET Occupation Type (Bubble Chart)

- Analyze how salaries differ across ONET occupation types.  
- **Aggregate Data**  
  - Compute median salary for each occupation in the ONET taxonomy.  
- **Visualize results**  
  - Create a **bubble chart** where:  
    - X-axis = `ONET_NAME` (occupation)  
    - Y-axis = `Median_Salary`  
    - Bubble size = `Job_Postings` (number of postings)  
  - Apply custom colors and font styles.  


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: Salary Analysis by Occupation (Auto-selected) – Bubble Chart

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
import plotly.express as px
from IPython.display import HTML, display
import re

# ---- Spark / source df
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("LightcastData").getOrCreate()

if any(t.name == "job_postings_clean" for t in spark.catalog.listTables()):
    base = spark.table("job_postings_clean")
else:
    base = (
        spark.read
        .option("header", "true").option("inferSchema", "true")
        .option("multiLine", "true").option("escape", "\"")
        .csv("data/lightcast_job_postings.csv")
    )

# ---- Choose salary column
salary_col = "Average_Salary" if "Average_Salary" in base.columns else (
    "SALARY_FROM" if "SALARY_FROM" in base.columns else "SALARY"
)
df = base.withColumn(salary_col, col(salary_col).cast(DoubleType()))

# ---- Find occupation-like columns and pick the best by distinct count
pattern = re.compile(r"(ONET|OCCUP|OCC|TITLE).*", re.IGNORECASE)
occ_candidates = [c for c in df.columns if pattern.match(c)]
# also include a few common fallbacks explicitly
for c in ["JOB_TITLE_NAME", "JOB_TITLE", "OCCUPATION_NAME", "OCC_TITLE", "ONET_NAME"]:
    if c in df.columns and c not in occ_candidates:
        occ_candidates.append(c)

distinct_counts = []
for c in occ_candidates:
    # count non-empty distinct values quickly
    cnt = (df.where(F.col(c).isNotNull() & (F.length(F.col(c)) > 0))
             .agg(F.approx_count_distinct(c).alias("n")).collect()[0]["n"])
    distinct_counts.append((c, cnt))

distinct_counts.sort(key=lambda x: x[1], reverse=True)
# pick the first with at least 5 distinct values; else fall back to the top one
occ_col, occ_distinct = (distinct_counts[0] if distinct_counts else (None, 0))
for c, n in distinct_counts:
    if n >= 5:
        occ_col, occ_distinct = c, n
        break

if not occ_col or occ_distinct <= 1:
    raise ValueError(
        f"No suitable occupation column with variety was found. "
        f"Checked: {distinct_counts}. Your dataset appears to have a single ONET occupation."
    )

# ---- Clean and explode multi-valued cells (comma/semicolon/pipe)
clean = df.withColumn(occ_col, F.trim(F.regexp_replace(col(occ_col), r"\s+", " ")))
split_col = F.split(F.regexp_replace(col(occ_col), r"\s*[,;/|]\s*", "|"), r"\|")
clean = clean.withColumn("Occupation", F.explode(split_col))
clean = clean.filter(F.col("Occupation").isNotNull() & (F.length(F.col("Occupation")) > 0))

# ---- Aggregate
agg = (
    clean.filter(col(salary_col).isNotNull() & (col(salary_col) > 0))
         .groupBy("Occupation")
         .agg(
             F.percentile_approx(col(salary_col), 0.5, 10000).alias("Median_Salary"),
             F.count(F.lit(1)).alias("Job_Postings")
         )
)

TOP_N = 20
pdf = (agg.orderBy(F.col("Job_Postings").desc()).limit(TOP_N)).toPandas()
pdf = pdf.sort_values("Job_Postings", ascending=False)

# ---- Plot
fig = px.scatter(
    pdf, x="Occupation", y="Median_Salary",
    size="Job_Postings", color="Job_Postings",
    color_continuous_scale="Viridis", size_max=80,
    hover_name="Occupation",
    labels={
        "Occupation": "Occupation Name",
        "Median_Salary": "Median Salary",
        "Job_Postings": "Number of Job Postings"
    },
    title=f"Salary Analysis by Occupation (Auto: {occ_col}, {occ_distinct} distinct)"
)
fig.update_layout(width=1800, height=800, margin=dict(l=40, r=40, t=90, b=320))
fig.update_xaxes(tickangle=45)
display(HTML(fig.to_html(include_plotlyjs="cdn", full_html=False)))

- **Explanation**: "This bubble chart illustrates the variation in median salaries across different occupations. The size of each bubble represents the number of job postings available for that role. What stands out from the data is that some positions, like Data Analyst, are in high demand, with a lot of job openings, but they offer moderate salaries. In contrast, more specialized roles, such as Data Engineer and Enterprise Architect, tend to have fewer job postings yet command much higher median salaries. This highlights an interesting trend in the job market where specialization can lead to greater financial rewards, even if the overall demand is lower."









# Salary by Education Level  

- Create two groups:  
  - **Associate’s or lower** (GED, Associate, No Education Listed)  
  - **Bachelor’s** (Bachelor’s degree)  
  - **Master’s** (Master’s degree)  
  - **PhD** (Doctorate, professional degree)  
- Plot scatter plots for each group using:  
  - `MAX_YEARS_EXPERIENCE` (with jitter)  
  - `Average_Salary`  
  - `OCCUPATION_NAME` (labels optional for hover) 

In [ ]:
#| echo: false
#| warning: false
#| fig-cap: Experience vs Salary by Education Level

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
from IPython.display import HTML, display
import plotly.express as px
import numpy as np

# --- Spark / source df
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("LightcastData").getOrCreate()

if any(t.name == "job_postings_clean" for t in spark.catalog.listTables()):
    df = spark.table("job_postings_clean")
else:
    df = (
        spark.read
        .option("header", "true").option("inferSchema", "true")
        .option("multiLine", "true").option("escape", "\"")
        .csv("data/lightcast_job_postings.csv")
    )

# --- Pick experience column robustly
exp_candidates = ["MAX_YEARS_EXPERIENCE", "YEARS_EXPERIENCE",
                  "YEARS_OF_EXPERIENCE", "MIN_YEARS_EXPERIENCE"]
exp_col = next((c for c in exp_candidates if c in df.columns), None)
if exp_col is None:
    raise ValueError(f"No experience column found. Tried: {exp_candidates}")

# --- Pick salary column robustly (prefer your computed Average_Salary)
if "Average_Salary" in df.columns:
    sal_col = "Average_Salary"
else:
    if "SALARY_FROM" in df.columns and "SALARY_TO" in df.columns:
        df = df.withColumn(
            "Average_Salary",
            (F.col("SALARY_FROM").cast(DoubleType()) + F.col("SALARY_TO").cast(DoubleType())) / 2.0
        )
        sal_col = "Average_Salary"
    elif "SALARY" in df.columns:
        sal_col = "SALARY"
    else:
        raise ValueError("No salary column found (looked for Average_Salary, SALARY_FROM/TO, SALARY).")

# Ensure salary is numeric
df = df.withColumn(sal_col, col(sal_col).cast(DoubleType()))

# --- Education grouping
edu_src = "EDUCATION_LEVELS_NAME" if "EDUCATION_LEVELS_NAME" in df.columns else None
if edu_src is None:
    raise ValueError("EDUCATION_LEVELS_NAME column not found.")

df = df.withColumn(
    "EDU_GROUP",
    F.when(F.lower(F.col(edu_src)).contains("phd"), "PhD")
     .when(F.lower(F.col(edu_src)).contains("master"), "Master’s")
     .when(F.lower(F.col(edu_src)).contains("bachelor"), "Bachelor")
     .otherwise("Associate or Lower")
)

# --- Optional: choose a hover occupation/title column if available
occ_candidates = ["OCCUPATION_NAME", "JOB_TITLE_NAME", "JOB_TITLE", "TITLE_RAW",
                  "ONET_NAME", "OCC_TITLE"]
occ_col = next((c for c in occ_candidates if c in df.columns), None)

# --- Build pandas dataframe for Plotly
cols = [exp_col, sal_col, "EDU_GROUP"] + ([occ_col] if occ_col else [])
pdf = (
    df.select(*cols)
      .where(col(sal_col).isNotNull() & (col(sal_col) > 0) & col(exp_col).isNotNull())
      .toPandas()
)

# Jitter experience a bit to reduce overplotting
rng = np.random.default_rng(42)
pdf["exp_jitter"] = pdf[exp_col].astype(float) + rng.uniform(-0.15, 0.15, size=len(pdf))

# Rename for pretty axes
pdf = pdf.rename(columns={sal_col: "Average Salary (USD)",
                          "exp_jitter": "Years of Experience"})

# --- Plot
hover = [occ_col] if occ_col else None
fig = px.scatter(
    pdf,
    x="Years of Experience",
    y="Average Salary (USD)",
    color="EDU_GROUP",
    hover_data=hover,
    opacity=0.7,
    title="Experience vs Salary by Education Level"
)

fig.update_layout(
    xaxis_title="Years of Experience",
    yaxis_title="Average Salary (USD)",
    legend_title="Education Group",
    width=1600, height=800,
    margin=dict(l=40, r=40, t=80, b=120)
)

from IPython.display import HTML
display(HTML(fig.to_html(include_plotlyjs="cdn", full_html=False)))

- **Explanation**: "This chart illustrates that salaries tend to rise with years of experience across all education levels, though there's significant variability, particularly in early to mid-career stages. Higher education levels, like Master’s and PhD, generally exhibit higher salary clusters compared to Associate’s and Bachelor’s degrees, although there is some overlap at various experience stages."




# Salary by Remote Work Type

- Split into three groups based on remote status: **Remote**, **Hybrid**, **Onsite** (includes blank/none).  
- Plot a **scatter** of `MAX_YEARS_EXPERIENCE` (with jitter) vs `Average_Salary`, colored by remote group.  
- Also create **salary histograms** for the three groups to compare distributions.  


In [ ]:
#| echo: false
#| warning: false
#| fig-cap: Experience vs Salary by Remote Work Type

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
from IPython.display import HTML, display
import plotly.express as px
import numpy as np

# --- Spark / source df
try:
    spark
except NameError:
    spark = SparkSession.builder.appName("LightcastData").getOrCreate()

if any(t.name == "job_postings_clean" for t in spark.catalog.listTables()):
    df = spark.table("job_postings_clean")
else:
    df = (
        spark.read
        .option("header", "true").option("inferSchema", "true")
        .option("multiLine", "true").option("escape", "\"")
        .csv("data/lightcast_job_postings.csv")
    )

# --- Experience column
exp_candidates = ["MAX_YEARS_EXPERIENCE", "YEARS_EXPERIENCE",
                  "YEARS_OF_EXPERIENCE", "MIN_YEARS_EXPERIENCE"]
exp_col = next((c for c in exp_candidates if c in df.columns), None)
if exp_col is None:
    raise ValueError(f"No experience column found. Tried: {exp_candidates}")

# --- Salary column (prefer Average_Salary)
if "Average_Salary" in df.columns:
    sal_col = "Average_Salary"
else:
    if "SALARY_FROM" in df.columns and "SALARY_TO" in df.columns:
        df = df.withColumn(
            "Average_Salary",
            (F.col("SALARY_FROM").cast(DoubleType()) + F.col("SALARY_TO").cast(DoubleType())) / 2.0
        )
        sal_col = "Average_Salary"
    elif "SALARY" in df.columns:
        sal_col = "SALARY"
    else:
        raise ValueError("No salary column found (looked for Average_Salary, SALARY_FROM/TO, SALARY).")

df = df.withColumn(sal_col, col(sal_col).cast(DoubleType()))

# --- Remote grouping
# Use existing REMOTE_GROUP if present; otherwise derive from REMOTE_TYPE_NAME
if "REMOTE_GROUP" in df.columns:
    df = df.withColumn("REMOTE_GROUP",
        F.when(F.col("REMOTE_GROUP").isin("Remote", "Hybrid", "Onsite"), F.col("REMOTE_GROUP"))
         .otherwise("Onsite")  # treat blanks/others as Onsite
    )
elif "REMOTE_TYPE_NAME" in df.columns:
    df = df.withColumn("REMOTE_GROUP",
        F.when(F.col("REMOTE_TYPE_NAME") == "Remote", "Remote")
         .when(F.col("REMOTE_TYPE_NAME") == "Hybrid", "Hybrid")
         .otherwise("Onsite")
    )
else:
    # if no remote info exists, create a single group to avoid failure
    df = df.withColumn("REMOTE_GROUP", F.lit("Onsite"))

# --- Optional hover info
hover_col = None
for c in ["JOB_TITLE_NAME", "TITLE_RAW", "OCCUPATION_NAME", "JOB_TITLE"]:
    if c in df.columns:
        hover_col = c
        break

# --- Build pandas DF for plots
cols = [exp_col, sal_col, "REMOTE_GROUP"] + ([hover_col] if hover_col else [])
pdf = (
    df.select(*cols)
      .where(col(sal_col).isNotNull() & (col(sal_col) > 0) & col(exp_col).isNotNull())
      .toPandas()
)

# jitter experience to reduce overplotting
rng = np.random.default_rng(42)
pdf["exp_jitter"] = pdf[exp_col].astype(float) + rng.uniform(-0.15, 0.15, size=len(pdf))
pdf = pdf.rename(columns={sal_col: "Average Salary (USD)", "exp_jitter": "Years of Experience"})

# --- Scatter plot
hover = [hover_col] if hover_col else None
fig_scatter = px.scatter(
    pdf,
    x="Years of Experience",
    y="Average Salary (USD)",
    color="REMOTE_GROUP",
    hover_data=hover,
    opacity=0.75,
    title="Experience vs Salary by Remote Work Type"
)
fig_scatter.update_layout(
    legend_title="Remote Work Type",
    width=1600, height=800,
    margin=dict(l=40, r=40, t=80, b=120)
)

# --- Histograms of salary by remote group (faceted)
fig_hist = px.histogram(
    pdf,
    x="Average Salary (USD)",
    color="REMOTE_GROUP",
    barmode="overlay",
    opacity=0.6,
    facet_col="REMOTE_GROUP",
    facet_col_spacing=0.06,
    title="Salary Distribution by Remote Work Type"
)
fig_hist.update_layout(
    width=1600, height=500,
    margin=dict(l=40, r=40, t=70, b=60),
    legend_title="Remote Work Type"
)

# --- Render only charts on the page
display(HTML(fig_scatter.to_html(include_plotlyjs="cdn", full_html=False)))
display(HTML(fig_hist.to_html(include_plotlyjs=False, full_html=False)))

- **Explanation**: "These charts reveal salary trends across onsite, remote, and hybrid roles. Most salaries cluster between $60k and $150k, but remote roles tend to lean towards the lower end, while onsite positions include more high-paying outliers. Hybrid roles follow a similar pattern. This indicates that remote work status isn’t the main factor influencing salary—job type and experience are likely more significant determinants."
